# Análise e limpeza de dados dos indicados ao Prêmio Pulitzer

Esta é uma análise exploratória e de limpeza de uma planilha contendo informações sobre livros indicados ao [Prêmio Pulitzer](https://www.pulitzer.org/).

Os dados desta planilha foram extraídos utilizando o pacote [`scrapereads`](https://github.com/guilhermecxe/scrapereads), um projeto de web scraping focado em dados de livros. As informações, especialmente as colunas relacionadas a IDs, foram obtidas do site [Goodreads](https://www.goodreads.com/) e extraídas em dia 22 de junho de 2024.

Devido à diversidade das obras elegíveis para o Pulitzer em diferentes categorias, é natural que os dados extraídos do Goodreads incluam apenas livros e outras categorias relacionadas a este tipo de obra. Consequentemente, a maioria das categorias relacionadas ao jornalismo, assim como as categorias *Music* e *Special Citation*, não estão presentes na base de dados utilizada neste estudo.

Por fim, informa-se que o objetivo deste projeto é normalizar os dados referentes às obras premiadas nas áreas que o Prêmio Pulitzer chama de "Letters & Drama". Embora o foco seja nas obras vencedoras, algumas obras apenas indicadas ao prêmio ainda podem constar na base de dados ao final deste processo.

## Sumário

- Configuração do ambiente
- Importação de dados
- Exploração inicial dos dados
- Formatação das colunas
- Correção das categorias de premiação
    - Distinção entre vencedores e indicados
    - Normalização de nomes de categorias
    - Tratamento de múltiplas indicações
    - Categoria *Biography or Autobiography*
    - Categoria *Letters Award*
- Identificação e remoção de registros duplicados
- Finalização

## Configuração do ambiente

In [1]:
# Importação dos pacotes necessários
import pandas as pd
import ast
import re

## Importação de dados

In [2]:
# Importando os dados
df = pd.read_excel('data/pulitzer/pulitzer.xlsx')

# Definindo as colunas principais para exibição
main_columns = ['id', 'title', 'author', 'prize']

# Exibindo uma amostra dos dados
print('Shape:', df.shape)
df[main_columns].head()

Shape: (1035, 12)


,id,title,author,prize
0,213009370,Primary Trust,Eboni Booth,Pulitzer Prize for Drama (2024)
1,77920745,A Day in the Life of Abed Salama: Anatomy of a...,Nathan Thrall,Pulitzer Prize for General Nonfiction (2024)
2,75527796,Tripas: Poems,Brandon Som,Pulitzer Prize for Poetry (2024)
3,61111247,Liliana's Invincible Summer: A Sister's Search...,Cristina Rivera Garza,Pulitzer Prize for Memoir or Autobiography (2024)
4,62039291,King: A Life,Jonathan Eig,Pulitzer Prize for Biography (2024)


## Exploração inicial dos dados

Aqui é feita a análise de valores ausentes, dos tipos de dados e das duplicidades.

In [3]:
# Exibindo a quantidade de valores nulos por coluna
df.isna().sum()

id                 0
editions_id        0
title              0
author             0
author_id          0
published_year     4
rating             0
ratings_count      0
reviews_count      0
genres             0
synopsis          17
prize              0
dtype: int64

É aceitável a falta de informações acerca da sinopse e do ano de publicação de algumas obras.

In [4]:
# Obtendo a quantidade de linhas duplicadas
df.duplicated().sum()

0

In [5]:
# Obtendo a quantidade de valores únicos por coluna
df.nunique()

id                1035
editions_id       1035
title             1025
author             868
author_id          868
published_year     112
rating             157
ratings_count      788
reviews_count      508
genres             860
synopsis          1012
prize              795
dtype: int64

É suspeito que hajam obras com o mesmo título e a mesma sinopse.

In [6]:
# Obtendo as obras que possuem títulos que se repetem
df[df['title'].duplicated(keep=False)][main_columns].sort_values('prize')

,id,title,author,prize
1029,344342,Selected Poems,Richard Hugo,Pulitzer Prize Nominee for Poetry (1980)
990,2982704,Collected Poems,Josephine Miles,Pulitzer Prize Nominee for Poetry (1984)
713,13069032,Collected Poems,Jack Gilbert,Pulitzer Prize Nominee for Poetry (2013)
544,790418,Selected Poems,Conrad Aiken,Pulitzer Prize for Poetry (1930)
538,17307602,The Collected Poems,Robert Frost,Pulitzer Prize for Poetry (1931)
429,6500113,Collected Poems,Marianne Moore,Pulitzer Prize for Poetry (1952)
417,75504,The Collected Poems,Wallace Stevens,Pulitzer Prize for Poetry (1955)
321,1012923,Collected Poems,James Wright,Pulitzer Prize for Poetry (1972)
289,336479,The Collected Poems,Howard Nemerov,Pulitzer Prize for Poetry (1978)
276,1649407,Selected Poems,Donald Justice,Pulitzer Prize for Poetry (1980)


Apesar da repetição nos títulos, não há uma multiplicidade redundante de obras, todas se referem a diferentes autores e diferentes premiações.

In [7]:
# Obtendo as obras que possuem sinopses que se repetem e ignorando sinopses nulas
df[df['synopsis'].duplicated(keep=False) & ~df['synopsis'].isna()][main_columns + ['synopsis']].sort_values('synopsis')

,id,title,author,prize,synopsis
498,3210254,The Road To Reunion 1865-1900,Paul H. Buck,Pulitzer Prize for History (1938),"Many of the earliest books, particularly those..."
499,2361549,Pedlar's Progress: The Life of Bronson Alcott,Odell Shepard,Pulitzer Prize for Biography or Autobiography ...,"Many of the earliest books, particularly those..."
525,2595796,John Hay: From Poetry to Politics,Tyler Dennett,Pulitzer Prize for Biography or Autobiography ...,"Many of the earliest books, particularly those..."
610,10984172,"Julia Ward Howe, 1819-1910",Laura Elizabeth Richards,Pulitzer Prize for Biography or Autobiography ...,"Many of the earliest books, particularly those..."
469,2192174,The American Leonardo: A Life of Samuel F.B. M...,Carleton Mabee,Pulitzer Prize for Biography or Autobiography ...,This scarce antiquarian book is a facsimile re...
600,2468675,The Old Road to Paradise,Margaret Widdemer,Pulitzer Prize for Poetry (1919),This scarce antiquarian book is a facsimile re...
604,6070378,Why Marry?,Jesse Lynch Williams,Pulitzer Prize for Drama (1918),This scarce antiquarian book is a facsimile re...
514,7042945,The Colonial Period of American History: The S...,Charles McLean Andrews,Pulitzer Prize for History (1935),This three-volume work won the 1935 Pulitzer P...
515,7702343,The Colonial Period of American History: The S...,Charles McLean Andrews,Pulitzer Prize for History (1935),This three-volume work won the 1935 Pulitzer P...


In [8]:
# Exibindo as sinopses que se repetiram
for synopsis in df[df['synopsis'].duplicated() & ~df['synopsis'].isna()]['synopsis'].unique():
    print(synopsis, '\n')

Many of the earliest books, particularly those dating back to the 1900s and before, are now extremely scarce and increasingly expensive. We are republishing these classic works in affordable, high quality, modern editions, using the original text and artwork. 

This three-volume work won the 1935 Pulitzer Prize in History. 

This scarce antiquarian book is a facsimile reprint of the original. Due to its age, it may contain imperfections such as marks, notations, marginalia and flawed pages. Because we believe this work is culturally important, we have made it available as part of our commitment for protecting, preserving, and promoting the world's literature in affordable, high quality, modern editions that are true to the original work. 



Apesar da repetição das sinopses, observa-se que dois destes textos podem descrever diferentes obras, não sendo necessário tomar providências para alterar estes dados.

No entanto, observa-se também a repetição da obra intitulada "The Colonial Period of American History". Vamos verificar a particularidade desta repetição.

In [9]:
# Obtendo o título completo dos registros repetidos da obra "The Colonial Period of American History"
df[df['title'].str.contains('The Colonial Period')][main_columns]['title'].tolist()

['The Colonial Period of American History: The Settlements, Volume II',
 'The Colonial Period of American History: The Settlements, Volume I']

Por não haver menção quanto ao volume no site dos premiados do Pulitzer, manteremos apenas o primeiro volume deste livro.

In [10]:
# Removendo o volume II de The Colonial Period of American History
index = df[df['title'] == 'The Colonial Period of American History: The Settlements, Volume II'].index[0]
df.drop(index, axis='index', inplace=True)

In [11]:
# Exibindo o tipo de cada coluna
df.dtypes

id                  int64
editions_id        object
title              object
author             object
author_id          object
published_year    float64
rating            float64
ratings_count       int64
reviews_count       int64
genres             object
synopsis           object
prize              object
dtype: object

Todas as colunas possuem os tipos esperados. O ano de publicação deve continuar como do tipo `float`, pois este campo possui valores nulos e estes podem ser representados como `float`, mas não podem como `int`.

## Formatação das colunas

É aqui onde adicionamos, alteramos e excluímos colunas.

In [12]:
# Transformando os gêneros em uma string onde eles são separados apenas por ", "
df['genres'] = df['genres'].apply(lambda genres: ', '.join(ast.literal_eval(genres)))

In [13]:
# Separando a categoria e o ano da premiação em colunas específicas e excluindo a coluna prize
df['prize_category'] = df['prize'].apply(lambda text: re.sub('\s*\([0-9]*\)', '', text))
df['prize_year'] = df['prize'].apply(lambda text: int(re.findall('\([0-9]*\)', text)[0].strip('()')))
df.drop('prize', axis='columns', inplace=True)

In [14]:
# Definindo novas colunas principais
main_columns = ['id', 'title', 'author', 'prize_category', 'prize_year']

## Correção das categorias de premiação

In [15]:
# Exibindo uma contagem de obras por categoria
df['prize_category'].value_counts().sort_index()

prize_category
Pulitzer Prize Nominee for Biography                                10
Pulitzer Prize Nominee for Biography or Autobiography               52
Pulitzer Prize Nominee for Drama                                    13
Pulitzer Prize Nominee for Fiction                                  90
Pulitzer Prize Nominee for General Nonfiction                       89
Pulitzer Prize Nominee for History                                  87
Pulitzer Prize Nominee for Memoir or Autobiography                   4
Pulitzer Prize Nominee for Poetry                                   79
Pulitzer Prize for  Biography or Autobiography                       1
Pulitzer Prize for Biography                                         9
Pulitzer Prize for Biography Finalist                                1
Pulitzer Prize for Biography or Autobiography                      106
Pulitzer Prize for Biography or Autobiography (tie)                  4
Pulitzer Prize for Drama                                      

É possível perceber que algumas categorias são repetidas e não estão agrupadas apenas por alguma diferença específica de nomenclatura.

### Distinção entre vencedores e indicados

In [16]:
# Exibindo novamente as categorias encontradas
df['prize_category'].value_counts().sort_index()

prize_category
Pulitzer Prize Nominee for Biography                                10
Pulitzer Prize Nominee for Biography or Autobiography               52
Pulitzer Prize Nominee for Drama                                    13
Pulitzer Prize Nominee for Fiction                                  90
Pulitzer Prize Nominee for General Nonfiction                       89
Pulitzer Prize Nominee for History                                  87
Pulitzer Prize Nominee for Memoir or Autobiography                   4
Pulitzer Prize Nominee for Poetry                                   79
Pulitzer Prize for  Biography or Autobiography                       1
Pulitzer Prize for Biography                                         9
Pulitzer Prize for Biography Finalist                                1
Pulitzer Prize for Biography or Autobiography                      106
Pulitzer Prize for Biography or Autobiography (tie)                  4
Pulitzer Prize for Drama                                      

In [17]:
# Criando uma coluna para indicar se o livro foi nomeado ou indicado ao prêmio e
# configurando todos os livros como vencedores, por enquanto
df['prize_status'] = 'winner'

In [18]:
# Alterando o status de um livro na premiação para indicado
# caso o "Nominee" seja mencionado no nome da categoria nesta base de dados
for i, row in df.iterrows():
    if 'Nominee' in row['prize_category']:
        df.at[i, 'prize_status'] = 'nominee'
        df.at[i, 'prize_category'] = row['prize_category'].replace('Nominee ', '')

### Normalização de nomes de categorias

In [19]:
# Corrigindo alguns nomes de categorias
index = df[df['prize_category'] == 'Pulitzer Prize for  Biography or Autobiography'].index[0]
df.at[index, 'prize_category'] = 'Pulitzer Prize for Biography or Autobiography'

index = df[df['prize_category'] == 'Pulitzer Prize for Drama Winner'].index[0]
df.at[index, 'prize_category'] = 'Pulitzer Prize for Drama'

index = df[df['prize_category'] == 'Pulitzer Prize for Special citation'].index[0]
df.at[index, 'prize_category'] = 'Pulitzer Prize for Special Citation'

In [20]:
# Acessando o livro finalista na categoria de Biografia para entender se ele é vencedor ou apenas indicado
df[df['prize_category'] == 'Pulitzer Prize for Biography Finalist'][main_columns]

,id,title,author,prize_category,prize_year
25,52768414,Stranger in the Shogun's City: A Japanese Woma...,Amy Stanley,Pulitzer Prize for Biography Finalist,2021


In [21]:
# Corrigindo o nome da categoria e o status
df.at[25, 'prize_category'] = 'Pulitzer Prize for Biography'
df.at[25, 'prize_status'] = 'nominee'

In [22]:
# Acessando livros que indicam que houve um empate para entender se eles são vencedores ou
# apenas indicados e de qual categoria
df[df['prize_category'] == 'Pulitzer Prize for Biography or Autobiography (tie)'][main_columns]

,id,title,author,prize_category,prize_year
402,3175653,George Washington (Vol 1): Young Washington,Douglas Southall Freeman,Pulitzer Prize for Biography or Autobiography ...,1958
495,6601286,Andrew Jackson: Portrait of a President,Marquis James,Pulitzer Prize for Biography or Autobiography ...,1938
496,7503630,The Life of Andrew Jackson,Marquis James,Pulitzer Prize for Biography or Autobiography ...,1938
499,2361549,Pedlar's Progress: The Life of Bronson Alcott,Odell Shepard,Pulitzer Prize for Biography or Autobiography ...,1938


In [23]:
# Corrigindo o nome da categoria do prêmio
df.at[402, 'prize_category'] = 'Pulitzer Prize for Biography'
df.at[496, 'prize_category'] = 'Pulitzer Prize for Biography'
df.at[499, 'prize_category'] = 'Pulitzer Prize for Biography'

# O livro "Andrew Jackson: Portrait of a President" é um volume do livro "The Life of Andrew Jackson",
# sendo que o vencedor do prêmio foi este último
df.drop(495, axis='index', inplace=True)

### Tratamento de múltiplas indicações

In [24]:
# Exibindo novamente as categorias encontradas
df['prize_category'].value_counts().sort_index()

prize_category
Pulitzer Prize for Biography                                23
Pulitzer Prize for Biography or Autobiography              159
Pulitzer Prize for Drama                                   106
Pulitzer Prize for Fiction                                 165
Pulitzer Prize for General Nonfiction                      153
Pulitzer Prize for General Nonfiction and for Biography      1
Pulitzer Prize for History                                 200
Pulitzer Prize for Illustrated Reporting                     1
Pulitzer Prize for International Reporting                   1
Pulitzer Prize for Letters Award                             1
Pulitzer Prize for Memoir or Autobiography                   6
Pulitzer Prize for Nonfiction                                1
Pulitzer Prize for Novel                                    26
Pulitzer Prize for Poetry                                  188
Pulitzer Prize for Special Citation                          2
Name: count, dtype: int64

In [25]:
# Acessando um livro que a categoria sinaliza duas indicações
df[df['prize_category'] == 'Pulitzer Prize for General Nonfiction and for Biography'][main_columns]

,id,title,author,prize_category,prize_year
13,58890953,His Name Is George Floyd: One Man's Life and t...,Robert Samuels,Pulitzer Prize for General Nonfiction and for ...,2023


A obra "His Name Is George Floyd: One Man’s Life and the Struggle for Racial Justice" realmente foi indicada a dois prêmios em 2023. Tendo sido vencedor na categoria *General Nonfiction* e finalista em *Biography*.

In [26]:
# Duplicando esta última obra em razão da duplicidade real de indicações
row_to_duplicate = pd.DataFrame(df.iloc[13]).transpose()
df = pd.concat([df, row_to_duplicate], ignore_index=True)
df[main_columns].tail(2)

,id,title,author,prize_category,prize_year
1032,22402749,Bless the Beasts and Children,Glendon Swarthout,Pulitzer Prize for Fiction,1970
1033,58890953,His Name Is George Floyd: One Man's Life and t...,Robert Samuels,Pulitzer Prize for General Nonfiction and for ...,2023


In [27]:
# Corrigindo o nome da categoria
df.at[13, 'prize_category'] = 'Pulitzer Prize for General Nonfiction'

# Corrigindo o nome da categoria e o status
df.at[1033, 'prize_category'] = 'Pulitzer Prize for Biography'
df.at[1033, 'prize_status'] = 'nominee'

### Categoria *Biography or Autobiography*

In [28]:
df['prize_category'].value_counts().sort_index()

prize_category
Pulitzer Prize for Biography                      24
Pulitzer Prize for Biography or Autobiography    159
Pulitzer Prize for Drama                         106
Pulitzer Prize for Fiction                       165
Pulitzer Prize for General Nonfiction            154
Pulitzer Prize for History                       200
Pulitzer Prize for Illustrated Reporting           1
Pulitzer Prize for International Reporting         1
Pulitzer Prize for Letters Award                   1
Pulitzer Prize for Memoir or Autobiography         6
Pulitzer Prize for Nonfiction                      1
Pulitzer Prize for Novel                          26
Pulitzer Prize for Poetry                        188
Pulitzer Prize for Special Citation                2
Name: count, dtype: int64

Existe realmente uma única categoria chamada *Memoir or Autobiography*. No entanto, não há uma categoria intitulada *Biography or Autobiography* ou até mesmo *Autobiography* (Fonte: https://www.pulitzer.org/prize-winners-categories)

In [29]:
# Obtendo uma amostragem dos livros da categoria Biography or Autobiography para verificar a qual
# categoria eles realmente pertencem
df[df['prize_category'] == 'Pulitzer Prize for Biography or Autobiography'].sample(n=5, random_state=0)[main_columns]

,id,title,author,prize_category,prize_year
93,4839382,The First Tycoon: The Epic Life of Cornelius V...,T.J. Stiles,Pulitzer Prize for Biography or Autobiography,2010
297,64682,A Prince of Our Disorder: The Life of T.E. Law...,John E. Mack,Pulitzer Prize for Biography or Autobiography,1977
557,2595812,The American Orchestra and Theodore Thomas,Charles Edward Russell,Pulitzer Prize for Biography or Autobiography,1928
320,20663740,Eleanor and Franklin,Joseph P. Lash,Pulitzer Prize for Biography or Autobiography,1972
694,20821221,"Stalin: Paradoxes of Power, 1878-1928",Stephen Kotkin,Pulitzer Prize for Biography or Autobiography,2015


Todos os livros acima pertencem à categoria *Biography*, logo, chego à decisão de definir todos os livros desta categoria como sendo, na verdade, da categoria *Biography*

In [30]:
df['prize_category'].replace('Pulitzer Prize for Biography or Autobiography', 'Pulitzer Prize for Biography', inplace=True)

### Categoria *Letters Award*

In [31]:
df['prize_category'].value_counts()

prize_category
Pulitzer Prize for History                    200
Pulitzer Prize for Poetry                     188
Pulitzer Prize for Biography                  183
Pulitzer Prize for Fiction                    165
Pulitzer Prize for General Nonfiction         154
Pulitzer Prize for Drama                      106
Pulitzer Prize for Novel                       26
Pulitzer Prize for Memoir or Autobiography      6
Pulitzer Prize for Special Citation             2
Pulitzer Prize for Illustrated Reporting        1
Pulitzer Prize for International Reporting      1
Pulitzer Prize for Letters Award                1
Pulitzer Prize for Nonfiction                   1
Name: count, dtype: int64

Não há um *Letters Awards*, então vamos verificar a qual categoria esta obra pertence.

In [32]:
df[df['prize_category'] == 'Pulitzer Prize for Letters Award'][main_columns]

,id,title,author,prize_category,prize_year
200,15196,Maus I: A Survivor's Tale: My Father Bleeds Hi...,Art Spiegelman,Pulitzer Prize for Letters Award,1992


Ela pertence à categoria *Special Citation* (fonte: https://www.pulitzer.org/prize-winners-by-year/1992).

In [33]:
df.at[200, 'prize_category'] = 'Pulitzer Prize for Special Citation'

### Categoria "Nonfiction"

A categoria definida apenas como *Nonfiction* também não existe. Vamos verificar a qual categoria este livro pertence.

In [34]:
df[df['prize_category'] == 'Pulitzer Prize for Nonfiction'][main_columns]

,id,title,author,prize_category,prize_year
342,12474,"The Armies of the Night: History as a Novel, T...",Norman Mailer,Pulitzer Prize for Nonfiction,1969


Ele pertence à categoria *General Nonfiction* (fonte: https://www.pulitzer.org/prize-winners-by-category/223).

In [35]:
df.at[342, 'prize_category'] = 'Pulitzer Prize for General Nonfiction'

## Verificação dos premiados ao longo dos anos

In [36]:
df['prize_category'].value_counts()

prize_category
Pulitzer Prize for History                    200
Pulitzer Prize for Poetry                     188
Pulitzer Prize for Biography                  183
Pulitzer Prize for Fiction                    165
Pulitzer Prize for General Nonfiction         155
Pulitzer Prize for Drama                      106
Pulitzer Prize for Novel                       26
Pulitzer Prize for Memoir or Autobiography      6
Pulitzer Prize for Special Citation             3
Pulitzer Prize for Illustrated Reporting        1
Pulitzer Prize for International Reporting      1
Name: count, dtype: int64

*Illustrated Reporting*, *International Reporting* e *Special Citation* não pertencen à classe *Letters & Drama*, então não será dada atenção a estas categorias.

In [37]:
# Verificando o maior e o menor ano correspondente à premiação de um livro em cada categoria
for category in df['prize_category'].unique():
    print(category)
    min_year = df[(df['prize_category'] == category) & (df['prize_status'] == 'winner')]['prize_year'].min()
    max_year = df[(df['prize_category'] == category) & (df['prize_status'] == 'winner')]['prize_year'].max()
    print('\t min year:', min_year, ', max_year:', max_year)

Pulitzer Prize for Drama
	 min year: 1918 , max_year: 2024
Pulitzer Prize for General Nonfiction
	 min year: 1962 , max_year: 2024
Pulitzer Prize for Poetry
	 min year: 1918 , max_year: 2024
Pulitzer Prize for Memoir or Autobiography
	 min year: 2023 , max_year: 2024
Pulitzer Prize for Biography
	 min year: 1917 , max_year: 2024
Pulitzer Prize for History
	 min year: 1917 , max_year: 2024
Pulitzer Prize for Fiction
	 min year: 1934 , max_year: 2024
Pulitzer Prize for Illustrated Reporting
	 min year: 2022 , max_year: 2022
Pulitzer Prize for Special Citation
	 min year: 1992 , max_year: 2007
Pulitzer Prize for International Reporting
	 min year: 1993 , max_year: 1993
Pulitzer Prize for Novel
	 min year: 1918 , max_year: 1945


Todos os anos limites estão em conformidade com as informações do site The Pulitzer Prizes, mas a categoria *Fiction* começou a ser premiada apenas em 1948.

In [38]:
# Verificando quais livros constam como premiados em Fiction antes de 1948
df[(df['prize_year'] < 1948) & (df['prize_category'] == 'Pulitzer Prize for Fiction')][main_columns]

,id,title,author,prize_category,prize_year
457,5527,All the King’s Men,Robert Penn Warren,Pulitzer Prize for Fiction,1947
502,40239971,"Autant en emporte le vent, tome 3",Margaret Mitchell,Pulitzer Prize for Fiction,1937
524,267478,Lamb in His Bosom,Caroline Miller,Pulitzer Prize for Fiction,1934


Todos estes livros, na verdade, venceram na categoria *Novel*.

In [39]:
df.at[457, 'prize_category'] = 'Pulitzer Prize for Novel'
df.at[502, 'prize_category'] = 'Pulitzer Prize for Novel'
df.at[525, 'prize_category'] = 'Pulitzer Prize for Novel'

In [40]:
# Verificando anos que não possuem vencedores em cada categoria
for category in df['prize_category'].unique():
    print(category)
    min_year = df[(df['prize_category'] == category) & (df['prize_status'] == 'winner')]['prize_year'].min()
    max_year = df[(df['prize_category'] == category) & (df['prize_status'] == 'winner')]['prize_year'].max()
    print('\t', end='')
    for year in range(min_year+1, max_year):
        if not year in df[df['prize_category'] == category]['prize_year'].to_list():
            print(year, end=', ')
    print()

Pulitzer Prize for Drama
	1919, 1933, 1942, 1944, 1947, 1951, 1958, 1963, 1964, 1966, 1968, 1972, 1974, 1986, 1997, 2006, 
Pulitzer Prize for General Nonfiction
	
Pulitzer Prize for Poetry
	1920, 1921, 
Pulitzer Prize for Memoir or Autobiography
	
Pulitzer Prize for Biography
	1962, 
Pulitzer Prize for History
	1919, 1930, 1984, 
Pulitzer Prize for Fiction
	1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1954, 1957, 1964, 1971, 1974, 1977, 
Pulitzer Prize for Illustrated Reporting
	
Pulitzer Prize for Special Citation
	1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 
Pulitzer Prize for International Reporting
	
Pulitzer Prize for Novel
	1920, 1934, 1941, 1946, 


Das categorias que não possuem vencedores em determinado ano, foi indetificado que as categorias e anos abaixo possuem os seguintes premiados:

- Drama 1958 - Look Homeward, Angel, by Ketti Frings (fonte: https://www.pulitzer.org/prize-winners-by-category/218)

- History 1930 - The Founding of the American Republic, Volume II: The War of Independence: American Phase, by Claude H. Van Tyne (oughton) (fonte: https://www.pulitzer.org/prize-winners-by-category/220)

In [41]:
# Verificando se algum dos livros mencionados acima constam em algum registro na base de dados
df[df['title'].str.contains('Homeward') | df['title'].str.contains('The War of Independence')][main_columns]

,id,title,author,prize_category,prize_year
230,824933,Look Homeward: A Life of Thomas Wolfe,David Herbert Donald,Pulitzer Prize for Biography,1988


In [42]:
# Obtendo as informações dos livros que ficaram ausentes
new_book1 = pd.DataFrame({
    'id': '240701',
    'editions_id': '233178',
    'title': 'Look Homeward, Angel: A Comedy-Drama in Three Acts',
    'author': 'Ketti Frings',
    'author_id': '140861.Ketti_Frings',
    'published_year': 1986,
    'rating': 3.73,
    'ratings_count': 83,
    'reviews_count': 10,
    'genres': "Plays, Drama, Theatre, Fiction",
    'synopsis': '''Comedy-Drama. Ketti Frings, from the novel by Thomas Wolfe.
        10 m., 8 f. 2 exts. w. inset. An authentic American classic, this powerful
        and vital play captures the sardonic humor and the grief, both private and
        universal, of Wolfe’s novel about a youth coming of age. Concentrating on
        the last third of Wolfe’s story, the play vividly portrays Eugene Gant,
        his mother who is obsessed by her material holdings and who maintains barriers
        against the love of her family, the father—a stonecutter imprisoned by his failures,
        and the brother who never breaks away. Winner of the Pulitzer Prize and the New York Critics’
        award as best play of the season. Liist price $6.25. (Royalty, $50-$40 from Samuel French)
        Slightly Restricted.''',
    'prize_category': 'Pulitzer Prize for Drama',
    'prize_year': 1958,
    'prize_status': 'winner'
}, index=[0])

new_book2 = pd.DataFrame({
    'id': '2643433',
    'editions_id': '2668169',
    'title': 'The War of Independence',
    'author': 'Claude H. Van Tyne',
    'author_id': '2502047.Claude_H_Van_Tyne',
    'published_year': 2001,
    'rating': 3.00,
    'ratings_count': 1,
    'reviews_count': 1,
    'genres': 'History',
    'synopsis': 'Winner of the 1930 Pulitzer Prize in History.',
    'prize_category': 'Pulitzer Prize for History',
    'prize_year': 1930,
    'prize_status': 'winner'
}, index=[0])

In [43]:
# Adicionando as novas obras à tabela
df = pd.concat([df, new_book1, new_book2], axis='index')

## Identificação e remoção de registros duplicados

In [44]:
# Verificando se há obras duplicadas. Isto está sendo feito ao analisar se há um linha da tabela
# com autor, ano de premiação e categoria de premiação repetidos
df[df.duplicated(['author', 'prize_year', 'prize_category'], keep=False)].sort_values('prize_year')[main_columns + ['ratings_count']]

,id,title,author,prize_category,prize_year,ratings_count
591,1641264,Miss Lulu Bett,Zona Gale,Pulitzer Prize for Drama,1921,164
590,7422491,Miss Lulu Bett: A Play,Zona Gale,Pulitzer Prize for Drama,1921,27
582,2431930,"The Supreme Court in United States History, Vo...",Charles Warren,Pulitzer Prize for History,1923,6
579,3792295,The Supreme Court in United States History,Charles Warren,Pulitzer Prize for History,1923,2
553,2066942,"Main Currents in American Thought, Vol. 3: The...",Vernon Louis Parrington,Pulitzer Prize for History,1928,20
555,1790242,"Main Currents in American Thought, Vol. 1: The...",Vernon Louis Parrington,Pulitzer Prize for History,1928,41
552,24631399,Main Currents in American Thought,Vernon Louis Parrington,Pulitzer Prize for History,1928,3
554,143304,"Main Currents in American Thought, Vol. 2: The...",Vernon Louis Parrington,Pulitzer Prize for History,1928,30
547,4222465,The Organization and Administration of the Uni...,Fred Albert Shannon,Pulitzer Prize for History,1929,2
545,10065723,The Organization and Administration of the Uni...,Fred Albert Shannon,Pulitzer Prize for History,1929,2


Para decidir qual registro manter na base, optou-se por aquele com o título mais similar ao encontrado no site The Pulitzer Prizes e, em caso de dúvida, manter o mais popular (com mais avaliações).

Algumas observações sobre os títulos na base em relação ao site The Pulitzer Prizes:
- "The Supreme Court in United States History" não menciona volume.
- "Main Currents in American Thought" menciona volume 2
- "My Experiences in the World War" não menciona volume
- "The Right Madness on Skye: Poems" foi indicado em 1981
- Não encontrei qualquer menção a "House of Light" de Mary Oliver ter vencido qualquer Pulitzer.

In [45]:
# Removendo registros
df.drop([590, 582, 553, 555, 552, 547, 530, 514, 515, 516, 502, 476, 429, 428, 395,
         396, 397, 398, 399, 402, 371, 275, 251, 199], axis='index', inplace=True)

In [46]:
# Alterando o ano de premiação
df.at[1026, 'prize_year'] = 1981

In [47]:
# Checando se ainda há registro repetidos
df[df.duplicated(['author', 'prize_year', 'prize_category'], keep=False)].sort_values('prize_year')

,id,editions_id,title,author,author_id,published_year,rating,ratings_count,reviews_count,genres,synopsis,prize_category,prize_year,prize_status


## Finalização

In [48]:
# Redefinindo a coluna de índice
df.reset_index(drop=True, inplace=True)

In [49]:
# Salvando os dados
df.to_excel('data/pulitzer/pulitzer_cleaned.xlsx', index=0)